In [1]:
!pip install rembg[gpu]

     ---------------------------------------- 47.7/47.7 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 296.5/296.5 kB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 38.1/38.1 MB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 60.9/60.9 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 6.5/6.5 MB 10.9 MB/s eta 0:00:00
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 82.0/82.0 kB 4.5 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     -------------------------------------- 137.0/137.0 MB 6.6 MB/s eta 0:00:00
  Using cached frozenlist-1.3.3-cp310-cp310-win_amd64.whl (33 kB)
     -------------------------------

In [40]:
from rembg import remove, new_session
from PIL import Image
import os
from tqdm import tqdm
from IPython.display import display

In [43]:
session = new_session("isnet-general-use")

background_fill_colors = list()
# xterm colors
background_fill_colors.append((0,0,0,255))
background_fill_colors.append((192,192,192,255))
for i in range(7):
    background_fill_colors.append((0 if i&1 else 128,0 if i&2 else 128,0 if i&4 else 128,255))
    background_fill_colors.append((0 if i&1 else 255,0 if i&2 else 255,0 if i&4 else 255,255))

In [75]:
def remove_background(source_root, output_root, item_seq):
    item_seq_imgs = os.listdir(os.path.join(source_root, item_seq))
    process_imgs = [item_seq_imgs[0], item_seq_imgs[-1]]
    output_path = os.path.join(output_root, item_seq)
    for fb_index, image_path in enumerate(process_imgs):
        image_name, image_ext = os.path.splitext(image_path)
        image = Image.open(os.path.join(source_root, item_seq, image_path))
        for index, value in tqdm(enumerate(background_fill_colors)): # 각 색상당
            bg_removed_image = remove(image, bgcolor=value)
            width, height = bg_removed_image.size
            cropped_image = bg_removed_image.crop(
                ((width-1500) // 2, 
                 (height-1500)//2, 
                 (width+1500)//2,
                 (height+1500)//2)
            )
#             display(bg_removed_image)
            for rot in range(0, 360, 18): # 20장의 샘플 이미지 생성
                rotated_image = cropped_image.rotate(angle=rot, fillcolor=value).resize((224,224))
                if not os.path.exists(output_path):
                    os.makedirs(output_path, exist_ok=True)
                rotated_image.convert('RGB').save(
                    os.path.join(
                        output_path,
                        f'{item_seq}_{fb_index}_c{index:02d}_r{rot:03d}{image_ext}'))

In [77]:
source_root = 'e:\\pill_image'
output_root = 'e:\\pill_image_augmented'
if not os.path.exists(output_root):
    os.makedirs(output_root, exist_ok=True)
item_seqs = [ x for x in os.listdir(source_root) if os.path.isdir(os.path.join(source_root, x)) ]
for item_seq in tqdm(item_seqs):
    remove_background(source_root, output_root, item_seq)


  0%|                                                                                         | 0/5088 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [29]:
os.path.join('/a', 'b/d','c')

'/a\\b/d\\c'